In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from matplotlib import pyplot as plt

In [ ]:
df=pd.read_csv('/content/row_cleaned.csv')
data=df.loc[:, ~df.columns.isin(['Sno', 'iso_code','country'])]
x = data.values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
data = pd.DataFrame(x_scaled)

data.to_csv('/content/normalized.csv')

In [ ]:
df_min_max_scaled = df.copy()

for column in df_min_max_scaled.iloc[:, 4:]:
    df_min_max_scaled[column] = (df_min_max_scaled[column] - df_min_max_scaled[column].min()) / (df_min_max_scaled[column].max() - df_min_max_scaled[column].min())


df_min_max_scaled.to_csv('/content/min-max-normalized.csv')

In [ ]:
def explore(row: float):
  if row >= 0.25:
    return 'H'
  else:
    return 'L'
a=df_min_max_scaled['co2_per_gdp'].tolist()
b=[]
for r in a:
  b.append(explore(r))

df_min_max_scaled['class']=b
print(df_min_max_scaled['class'])
df_min_max_scaled.to_csv('/content/min-max-normalized.csv')

In [ ]:
classa=df_min_max_scaled.loc[df_min_max_scaled['class']=='high']
Mean=np.mean(classa , axis=0)
SD=np.std(classa, axis=0)
print(Mean)
print(SD)

In [ ]:
classb=df_min_max_scaled.loc[df_min_max_scaled['class']=='low']
Mean1=np.mean(classb , axis=0)
SD1=np.std(classa, axis=0)
print(Mean1)
print(SD1)

In [ ]:
dist=np.linalg.norm(Mean-Mean1)
print(dist)

In [ ]:
np.histogram(df_min_max_scaled['consumption_co2_per_capita'],bins=[0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1],density=True)
m=df_min_max_scaled['consumption_co2_per_capita'].mean()
var=df_min_max_scaled['consumption_co2_per_capita'].var()
print("Mean:",m)
print("Var:",var)

In [ ]:
plt.hist(df_min_max_scaled['consumption_co2_per_capita'],bins=[0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1])
plt.show()

In [ ]:
from scipy.spatial import distance
a=df_min_max_scaled.iloc[18,3:-1]
b=df_min_max_scaled.iloc[19,3:-1]
ans=[]
for p in range(1,11):
  ans.append(distance.minkowski(a,b,p))
print(ans)
plt.plot(ans)
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
z=df_min_max_scaled.iloc[:,:-1]

a_train, a_test, b_train, b_test = train_test_split(z, df_min_max_scaled['class'], test_size=0.3)
print(a_test)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
a=a_train.iloc[:,3:-1]
neigh.fit(a, b_train)

In [ ]:
a1=a_test.iloc[:,3:-1]
s1=neigh.score(a, b_train)
s2=neigh.score(a1, b_test)
print(s1)
print(s2)

In [ ]:
neigh.predict(a_test.iloc[30:31,3:-1])


In [ ]:
neighbors = np.arange(1, 11)
train_accuracy = np.empty(len(neighbors))
test_accuracy = np.empty(len(neighbors))
  
for i, k in enumerate(neighbors):
    neigh = KNeighborsClassifier(n_neighbors=k)
    a=a_train.iloc[:,3:-1]
    neigh.fit(a, b_train)
      
    train_accuracy[i] = neigh.score(a_train.iloc[:,3:-1], b_train)
    test_accuracy[i] = neigh.score(a_test.iloc[:,3:-1], b_test)
  
plt.plot(neighbors, test_accuracy, label = 'Testing dataset Accuracy')
plt.plot(neighbors, train_accuracy, label = 'Training dataset Accuracy')
  
plt.legend()
plt.alabel('n_neighbors')
plt.blabel('Accuracy')
plt.show()